In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold

from sklearn.preprocessing import StandardScaler

from scipy import stats

%matplotlib inline

I:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [238]:
train_data = pd.read_csv('house_price/train.csv')
train_data = train_data.drop(['Id'] , axis = 1)

In [239]:
train_data.loc[train_data.GrLivArea>4500,'GrLivArea'].index #找离群点
train_data = train_data.drop([523 , 1298]) #上面已经把1298删除了
#注意删除前面的离群值后 后面的样本序号会变

In [240]:
#对saleprice进行除偏
train_data.SalePrice = train_data.SalePrice.apply(np.log1p)

In [241]:
train_data.Alley = train_data.Alley.fillna('None')#
train_data.MasVnrType = train_data.MasVnrType.fillna('None')#

train_data.BsmtQual = train_data.BsmtQual.fillna('None')#
train_data.BsmtCond = train_data.BsmtCond.fillna('None')#
train_data.BsmtExposure = train_data.BsmtExposure.fillna('None')#
train_data.BsmtFinType1 = train_data.BsmtFinType1.fillna('None')#
train_data.BsmtFinType2 = train_data.BsmtFinType2.fillna('None')#

train_data.Electrical = train_data.Electrical.fillna(train_data.Electrical.value_counts().index[0])
train_data.FireplaceQu = train_data.FireplaceQu.fillna('None')#
train_data.GarageType = train_data.GarageType.fillna('None')#
train_data.GarageFinish = train_data.GarageFinish.fillna('None')#
train_data.GarageQual = train_data.GarageQual.fillna('None')#
train_data.GarageCond = train_data.GarageCond.fillna('None')#
train_data.PoolQC = train_data.PoolQC.fillna('None')#
train_data.Fence = train_data.Fence.fillna('None')#
train_data.MiscFeature = train_data.MiscFeature.fillna('None')#

train_data.LotFrontage = train_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))#
train_data.MasVnrArea = train_data.MasVnrArea.fillna(0)#
train_data.GarageYrBlt = train_data.GarageYrBlt.fillna(0)

train_data.GarageYrBlt = train_data.GarageYrBlt.fillna(0)#
train_data.GarageArea = train_data.GarageArea.fillna(0)#
train_data.GarageCars = train_data.GarageCars.fillna(0)#


for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    train_data[col] = train_data[col].fillna(0)

train_data.MSZoning = train_data.MSZoning.fillna(train_data.MSZoning.value_counts().index[0])



In [243]:
train_data = train_data.drop('Utilities' , axis=1)

In [244]:
train_data.Functional = train_data.Functional.fillna('Typ')

In [245]:
train_data.Electrical = train_data.Electrical.fillna(train_data.Electrical.value_counts().index[0])
train_data.KitchenQual = train_data.KitchenQual.fillna(train_data.KitchenQual.value_counts().index[0])


In [246]:
train_data.Exterior1st = train_data.Exterior1st.fillna(train_data.Exterior1st.value_counts().index[0])
train_data.Exterior2nd = train_data.Exterior2nd.fillna(train_data.Exterior2nd.value_counts().index[0])


In [247]:
train_data.SaleType = train_data.SaleType.fillna(train_data.SaleType.value_counts().index[0])


In [248]:
train_data.MSSubClass = train_data.MSSubClass.fillna('None')

In [249]:
#继续eda
#将数字变为str类型 为转换为categorical 类型做准备
train_data.MSSubClass = train_data.MSSubClass.apply(str)
train_data.OverallCond = train_data.OverallCond.astype(str)

train_data.YrSold = train_data.YrSold.astype(str)
train_data.MoSold = train_data.MoSold.astype(str)

In [250]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for i in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(train_data[i].values)) 
    train_data[i] = lbl.transform(list(train_data[i].values))



In [251]:
train_data.shape

(1458, 79)

In [252]:
train_data['TotalSF'] = train_data['TotalBsmtSF'] + train_data['1stFlrSF'] + train_data['2ndFlrSF']

In [253]:
train_label = train_data.SalePrice
train_data = train_data.drop('SalePrice' , axis = 1)

In [254]:
#=================
#boxcox除偏
numerical_features = train_data.dtypes[train_data.dtypes != object].index #含有预测值 SalePrice

skewed_feats = train_data[numerical_features].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
MiscVal,24.434913
PoolArea,15.932532
LotArea,12.560986
3SsnPorch,10.286510
LowQualFinSF,8.995688
LandSlope,4.805032
KitchenAbvGr,4.480268
BsmtFinSF2,4.247550
ScreenPorch,4.114690
BsmtHalfBath,4.095895


In [255]:
from scipy import special

for i in skewness[np.abs(skewness)>0.75].index:
    train_data[i] = special.boxcox1p(train_data[i] , 0.15)


In [256]:
train_data = pd.get_dummies(train_data)

In [257]:
train_data.shape

(1458, 220)

In [ ]:
#==============
#==============

#==============
#测试区域

In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

forest = RandomForestRegressor(n_estimators=100)
linear = LinearRegression()
svr = SVR()
dec_tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()
ridge = Ridge(alpha=3)
lasso = Lasso()

classifier = {'random_forest':forest , 'LR':linear , 'SVM':svr , 'decison_tree':dec_tree , 'KNN':knn , 'ridge':ridge , 'lasso':lasso}
#kf = KFold(train_data.shape[0] , 10 , True)

In [104]:
def rmse(model , data , label):
    #cv=5 分为5folds
    #在这里会对模型进行训练 
    return np.sqrt(-cross_val_score(model, data, label, scoring="neg_mean_squared_error", cv = 10))

In [258]:
vali_index = list(train_data.columns)

In [260]:
best_model = _
smallest_rmse = 1.0
for model_name , model in classifier.items():
    result = np.average(rmse(model , train_data[vali_index] , train_label))
    if result<smallest_rmse:
        smallest_rmse = result
        best_model = model_name
        
    print(model_name , result)

print('\nbest:',best_model , smallest_rmse)
#真大

random_forest 0.13699586443029427
LR 291928935.76624644
SVM 0.14804276072673941
decison_tree 0.20516579000620636
KNN 0.2320032424806835
ridge 0.11349391596739941
lasso 0.39884066356618436

best: ridge 0.11349391596739941


In [687]:
from sklearn.model_selection import GridSearchCV

def grid_search_scoring(model , data , label):
    return np.average(rmse(model , data , label))

In [688]:
param_grid={'alpha':[0.5 , 1.0 , 1.5 , 2.0 , 2.5 , 3.0]}
kfold = KFold(train_data.shape[0] , 10 , True)
grid = GridSearchCV(estimator=ridge , param_grid=param_grid , scoring=grid_search_scoring , cv=kfold)

In [689]:
grid.fit(train_data[vali_index] , train_data.SalePrice)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1458, n_folds=10, shuffle=True, random_state=None),
       error_score='raise',
       estimator=Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function grid_search_scoring at 0x00000239A7E94158>,
       verbose=0)

In [691]:
grid.best_score_

0.189017025638297

In [225]:
#================
#================
#投入使用
ridge.fit(train_data[vali_index] , train_data.SalePrice)

Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [232]:
#进行预测
#生成结果
result = ridge.predict(test_data[vali_index])
result = np.exp(result)
result_csv = pd.DataFrame({'Id':test_id , 'SalePrice':np.exp(result)})
result_csv.to_csv('submit2.csv' , index=False)

In [230]:
result = np.exp(result)

In [231]:
result

array([22.06242109, 22.70301388, 23.10653168, ..., 22.99333764,
       21.95771734, 23.59991223])